<a href="https://colab.research.google.com/github/Cristian2820/Senales-y-sistemas/blob/main/Parciales/Parcial%203/Dashboard_Parcial_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Punto 1

Este código se utiliza para instalar las bibliotecas y herramientas necesarias para desarrollar una aplicación web en Streamlit que simula un sistema de control. La línea `!pip install streamlit -q` instala la biblioteca Streamlit, que permite crear aplicaciones web interactivas de manera sencilla. La línea `!npm install localtunnel` instala Localtunnel, una herramienta que expone aplicaciones locales a una URL pública. Luego, `!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz` instala o reinstala yt-dlp, una herramienta para descargar videos de diversas plataformas. Además, `!pip install pyngrok` instala Pyngrok, que permite crear túneles seguros para exponer aplicaciones locales a Internet. Por último, `!pip install sympy --upgrade` actualiza la biblioteca Sympy, que proporciona funciones para matemáticas simbólicas, incluyendo un módulo de control para sistemas dinámicos.



In [43]:
!pip install streamlit -q #instalación de librerías
!npm install localtunnel
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install pyngrok
!pip install sympy --upgrade #actualizar sympy para usar módulo de control


up to date, audited 23 packages in 534ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Brotli-1.1.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached mutagen-1.47.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached pycryptodomex-3.20.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached websoc

El código es un script de Python que utiliza Streamlit para crear una aplicación web que simula un sistema de control de segundo orden. Importa bibliotecas necesarias como streamlit, numpy, matplotlib.pyplot y sympy para realizar cálculos simbólicos y graficación. La aplicación establece un título y una breve descripción de su funcionalidad, inicializa variables simbólicas para la masa (m), el coeficiente de amortiguamiento (c) y la rigidez (k), y crea controles deslizantes en la barra lateral para que el usuario ajuste estos valores. Calcula la ganancia, la frecuencia natural, el coeficiente de amortiguamiento y los polos del sistema, mostrando además los equivalentes de resistencia, inductancia y capacitancia. Finalmente, genera gráficos que ilustran el diagrama de polos y ceros, el diagrama de Bode, la respuesta al impulso y la respuesta al escalón, facilitando así la comprensión del comportamiento de sistemas de control.

In [47]:
%%writefile Punto1.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
from sympy.physics.control.lti import TransferFunction
from sympy.physics.control.control_plots import pole_zero_plot, bode_plot, impulse_response_plot, step_response_plot

# Título de la aplicación
st.title('Simulación de Sistema de Control de Segundo Orden con Ajuste de Parámetros')

# Descripción o texto en la app
st.write("Simula un sistema de control de segundo orden ajustando m, c y k. ")


# Inicializar variables en Sympy
sym.init_printing()
t, m, c, k = sym.symbols('t m c k', real=True, positive=True)
s = sym.symbols('s', complex=True)

# Crear sliders para los valores de m, c, y k
m_val = st.sidebar.slider('Masa (m)', min_value=0.1, max_value=10.0, value=4.0, step=0.1)
c_val = st.sidebar.slider('Amortiguamiento (c)', min_value=0.0, max_value=5.0, value=1.0, step=0.1)
k_val = st.sidebar.slider('Rigidez (k)', min_value=0.1, max_value=10.0, value=1.0, step=0.1)

# Definir los coeficientes de la ecuación
a2 = m
a1 = c
ao = k

Ko = 1/ao  # Ganancia circuito forma canónica
wn = sym.sqrt(ao/a2)
xi = a1/(2*sym.sqrt(ao*a2))

# Función de transferencia
H = Ko * wn**2 / (s**2 + 2 * xi * wn * s + wn**2)

# Calcular los polos
p1 = -xi*wn + 1j*wn*sym.sqrt(1-xi**2)
p2 = -xi*wn - 1j*wn*sym.sqrt(1-xi**2)

# Reemplazar los valores en las ecuaciones
mck = {m: m_val, c: c_val, k: k_val}

# Mostrar los resultados en la aplicación
st.write(f"Polos del sistema:")
st.write(f"p1 = {p1.subs(mck)}")
st.write(f"p2 = {p2.subs(mck)}")
st.write(f"Coeficiente de amortiguamiento (ξ) = {xi.subs(mck)}")
st.write(f"Frecuencia natural (ωn) = {wn.subs(mck)}")

# Calcular los equivalentes RLC
R_value = c_val  # Resistencia es igual al coeficiente de amortiguamiento
L_value = m_val  # Inductancia es igual a la masa
C_value = 1 / k_val  # Capacitancia es inversa de la rigidez

st.write("### Equivalentes RLC:")
st.write(f" - Resistencia (R): {R_value:.2f} Ω")
st.write(f" - Inductancia (L): {L_value:.2f} H")
st.write(f" - Capacitancia (C): {C_value:.2f} F")

# Gráficas
st.write("### Gráficas del sistema")
num, den = sym.fraction(H.subs(mck))
tf1 = TransferFunction(num, den, s)

# Gráfica de polos y ceros
st.write("#### Diagrama de polos y ceros")
fig_pz, ax_pz = plt.subplots()
pole_zero_plot(tf1, ax=ax_pz)
st.pyplot(fig_pz)

# Bode Plot
st.write("#### Diagrama de Bode")
fig_bode, (mag_ax, phase_ax) = plt.subplots(2, 1)
bode_plot(tf1)  # Removemos 'ax' ya que lo maneja internamente
st.pyplot(fig_bode)

# Impulse Response Plot
st.write("#### Respuesta al impulso")
fig_impulse, ax_impulse = plt.subplots()
impulse_response_plot(tf1, upper_limit=5/(xi.subs(mck)*wn.subs(mck)))
st.pyplot(fig_impulse)

# Step Response Plot
st.write("#### Respuesta al escalón")
fig_step, ax_step = plt.subplots()
step_response_plot(tf1, upper_limit=5/(xi.subs(mck)*wn.subs(mck)))
st.pyplot(fig_step)


Overwriting Punto1.py


El código token="2lmAC6DGxB9xF06bq6Js7K7IZQK_5T6aE4rBZBp6hDqefow8M" asigna un valor de cadena (string) a la variable token. Este token se utiliza para autenticar y autorizar la creación de la aplicación en Streamlit, permitiendo el acceso a servicios como ngrok.

In [48]:
token="2lmAC6DGxB9xF06bq6Js7K7IZQK_5T6aE4rBZBp6hDqefow8M"

Este código importa la biblioteca ngrok para crear un túnel que expone la aplicación de Streamlit al público. Primero, configura el token de autenticación mediante ngrok.set_auth_token(token), lo que permite el acceso a la cuenta de ngrok del usuario. Luego, inicia el servidor de Streamlit en segundo plano en el puerto 5028 usando !nohup streamlit run Punto1.py --server.port 5028 &. A continuación, establece un túnel HTTP con ngrok.connect(addr='5028', proto='http', bind_tls=True), lo que permite el acceso seguro a la aplicación a través de HTTPS. Finalmente, imprime la URL pública generada para acceder a la aplicación de Streamlit desde cualquier lugar.

In [49]:
from pyngrok import ngrok

# # Set authentication token (unique per user)
ngrok.set_auth_token(token)

# # Start Streamlit server on a specific port
!nohup streamlit run Punto1.py --server.port 5028 &

# # Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5028', proto='http', bind_tls=True)

# # Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://d6f8-35-222-123-212.ngrok-free.app
